In [7]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [8]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"
#define NUM_GPIOS_PMOD_CONN 8

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1"); //simple error checking to ensure pin is either 0 or 1
    }
    gpio pin_out = gpio_open(pin); //declare a gpio object called pin_out. Use the open function to open it as a GPIO pin.
    gpio_set_direction(pin_out, GPIO_OUT); //Set the direction of the newly-defined pin_out gpio object to be an output.
    gpio_write(pin_out, val); //write the requested value by the function call to the pin.
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin); //declare a gpio object called pin_in. Use the open function to open it as a GPIO pin.
    gpio_set_direction(pin_in, GPIO_IN); //Set the direction of the newly-defined pin_in gpio object to be an input.
    return gpio_read(pin_in); //read the value currently on this GPIO pin
}

//Function to reset all eight GPIO pins on the PMODB connector:
void resetAllGPIOPMODB(void) {
    for (unsigned int gpio_ctr = 0; gpio_ctr < NUM_GPIOS_PMOD_CONN; ++gpio_ctr) write_gpio(gpio_ctr, 0);
}

In [3]:
### Question 1: Reset all GPIO pins, play with colors ###

#Define RGB pin #s:
blue_pin = 1
green_pin = 2
red_pin = 3

#Reset all:
resetAllGPIOPMODB()
time.sleep(0.5);

#Blue:
write_gpio(blue_pin, 1);
time.sleep(1.0);
write_gpio(blue_pin, 0);
time.sleep(1.0);

#Green:
write_gpio(green_pin, 1);
time.sleep(1.0);
write_gpio(green_pin, 0);
time.sleep(1.0);

#Red:
write_gpio(red_pin, 1);
time.sleep(1.0);
write_gpio(red_pin, 0);
time.sleep(1.0);

#Turquoise:
write_gpio(blue_pin, 1);
write_gpio(green_pin, 1);
time.sleep(1.0);
write_gpio(blue_pin, 0);
write_gpio(green_pin, 0);
time.sleep(1.0);

#Purple:
write_gpio(blue_pin, 1);
write_gpio(red_pin, 1);
time.sleep(1.0);
write_gpio(blue_pin, 0);
write_gpio(red_pin, 0);
time.sleep(1.0);

#Light Green:
write_gpio(green_pin, 1);
write_gpio(red_pin, 1);
time.sleep(1.0);
write_gpio(green_pin, 0);
write_gpio(red_pin, 0);
time.sleep(1.0);

#All:
write_gpio(green_pin, 1);
write_gpio(red_pin, 1);
write_gpio(blue_pin, 1);
time.sleep(1.0);
write_gpio(green_pin, 0);
write_gpio(red_pin, 0);
write_gpio(blue_pin, 0);

KeyboardInterrupt: 

In [88]:
### Questions 2-4: PWM ###

#Define RGB pin #s:
blue_pin = 1
green_pin = 2
red_pin = 3

resetAllGPIOPMODB() #run reset method
pin = green_pin #green looks the brightest at logic high
freq_Hz = 55
duty_cycle = 10
if duty_cycle > 100 or duty_cycle < 0:
    print("Invalid duty cycle entered. Enter DC between 0 and 100 percent (integer) inclusive.")
elif duty_cycle == 0:
    write_gpio(pin, 0) #needed due to finite time it takes to run the time.sleep() method, despite the argument being 0.0sec
elif duty_cycle == 100:
    write_gpio(pin, 1) #From what I saw, this if clause is not really needed as the OFF blip was imperceptible when running the else clause below with duty_cycle==100
else:
    period_sec = 1.0/freq_Hz
    sleep_time_ON = (duty_cycle/100)*period_sec
    print(str(sleep_time_ON) + " seconds") #debug statement
    sleep_time_OFF = (1-duty_cycle/100)*period_sec
    print(str(sleep_time_OFF) + " seconds") #debug statement
    while 1:
        write_gpio(pin, 1)
        time.sleep(sleep_time_ON)
        write_gpio(pin, 0)
        time.sleep(sleep_time_OFF)

0.0018181818181818182 seconds
0.016363636363636365 seconds


KeyboardInterrupt: 

In [119]:
### Question 5: Brightness Plot ###

#Define RGB pin #s:
blue_pin = 1
green_pin = 2
red_pin = 3

resetAllGPIOPMODB() #run reset method
pin = green_pin #green looks the brightest at logic high
freq_Hz = 55
period_sec = 1.0/freq_Hz

for ctr in range(19): #start at 5%, go to 95% in increments of 5%.
    duty_cycle = 5*(ctr+1)
    #print(duty_cycle)
    sleep_time_ON = (duty_cycle/100)*period_sec
    #print(sleep_time_ON)
    sleep_time_OFF = (1-duty_cycle/100)*period_sec
    #print(sleep_time_OFF)
    print(ctr*duty_cycle)
    
    #There are 55 periods of 55Hz in 1s, so run 55 iterations of PWM:
    for i in range(54):
        write_gpio(pin, 1)
        time.sleep(sleep_time_ON)
        write_gpio(pin, 0)
        time.sleep(sleep_time_OFF)

write_gpio(pin, 1)

0
10
30
60


KeyboardInterrupt: 

In [ ]:
### Question 6: RGB PWM System with On/Off Button ###
import asyncio

duty_cycle = 25 #fixed in this part
freq_Hz = 55 #found in previous sections
period_sec = 1/freq_Hz
sleep_time_ON = (duty_cycle/100)*period_sec
sleep_time_OFF = (1-duty_cycle/100)*period_sec
btns = base.btns_gpio
resetAllGPIOPMODB() #run reset method

#Define RGB pin #s:
blue_pin = 1
green_pin = 2
red_pin = 3

pin = red_pin #red to start, per instructions
cond = True

async def pwm_flash():
    global pin, ON_OFF, cond
    while cond: 
        for ctr in range(freq_Hz): #LED on second. Must go through on/off cycle freq_Hz times for the LED to be on for 1 second
            write_gpio(pin, 1)
            await asyncio.sleep(sleep_time_ON)
            write_gpio(pin, 0)
            await asyncio.sleep(sleep_time_OFF)
        await asyncio.sleep(1.0) #LED off second.
       
    
async def get_btns(_loop): #look if any buttons were pressed
    global pin, ON_OFF, cond
    while cond:
        await asyncio.sleep(0.01)
        if btns.read() == 1:
            pin = red_pin
            write_gpio(green_pin, 0)
            write_gpio(blue_pin, 0)
            ON_OFF = 1
            #cond = True
        elif btns.read() == 2:
            pin = green_pin
            write_gpio(red_pin, 0)
            write_gpio(blue_pin, 0)
            ON_OFF = 1
            #cond = True
        elif btns.read() == 4:
            pin = blue_pin
            write_gpio(green_pin, 0)
            write_gpio(red_pin, 0)
            #cond = True
        elif btns.read() == 8:
            pin = 4
            write_gpio(green_pin, 0)
            write_gpio(red_pin, 0)
            write_gpio(blue_pin, 0)
            #cond = True

loop = asyncio.new_event_loop()
loop.create_task(pwm_flash())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()        
print("Done.")